In [1]:
%reload_ext autoreload
%autoreload 2

from MTPHandler.core import Plate
from MTPHandler.readers import read_magellan

from EnzymePynetics.core import Estimator

## Standard curve for ABTS radical

In [2]:
standard_plate = Plate.from_reader(
    reader=read_magellan,
    path="laccase_kinetics/abtscorrelation_rawdata.xlsx",
    ph=3,
    wavelength=420,
)

standard_plate.visualize(wavelengths=420, zoom=False)

In [3]:
radical = standard_plate.add_reactant(
    id="abts_radical", name="ABTS radical", constant=False
)

buffer = standard_plate.add_reactant(id="buffer", name="Buffer", constant=True)

In [4]:
standard_plate.assign_species(
    species=buffer, conc_unit="mM", init_conc=50, to="all")


standard_plate.assign_species_from_spreadsheet(
    species=radical,
    conc_unit="mmol / l",
    path="laccase_kinetics/abtscorrelation_rawdata.xlsx",
    sheet_name="Well assignment",
)

Assigned Buffer to all wells.


In [5]:
standard_plate.blank_species(buffer, 420)

Mean absorption of Buffer at 50.0 mM: -0.046035714285714284
Blanked 62 wells containing Buffer.


In [6]:
standard = standard_plate.calibrate(radical, 420, cutoff=4)

In [7]:
standard.fit_models()

,AIC,R squared,RMSD
Model Name,,,
cubic,-26,0.6434,0.7621
quadratic,20,0.1896,1.1488
linear,48,-0.3642,1.4905


In [8]:
standard.visualize()

In [9]:
# Save the best model for concentration calculation
standard = standard.save_model("cubic")

## Process kinetic data

In [10]:
plate_woTween = Plate.from_reader(
    reader=read_magellan,
    path="laccase_kinetics/Laccase_kinetic_woTween_Rawdata.xlsx",
    ph=3,
    wavelength=420,
)

plate_Tween = Plate.from_reader(
    reader=read_magellan,
    path="laccase_kinetics/Laccase_kinetic_wthTween_Rawdata.xlsx",
    ph=3,
    wavelength=420,
)

plates = [plate_woTween, plate_Tween]

plate_Tween.visualize(wavelengths=420, zoom=False)

In [11]:
for plate in plates:
    # Define laccase
    laccase = plate.add_protein(
        id="p0",
        name="Trametes versicolor laccase",
        constant=True,
        uniprotid="O13448",
        sequence="MSRFHSLLAFVVASLTAVAHAGIGPVADLTITNAAVSPDGFSRQAVVVNGGTPGPLITGNMGDRFQLNVIDNLTNHTMLKSTSIHWHGFFQKGTNWADGPAFINQCPISSGHSFLYDFQVPDQAGTFWYHSHLSTQYCDGLRGPFVVYDPNDPAADLYDVDNDDTVITLVDWYHVAANVGPAFPLGADATLINGKGRSPSTTTADLSVISVTPGKRYRFRLVSLSCDPNYTFSIDGHNMTIIETDSINTAPLVVDSIQIFAAQRYSFVLEANQAVDNYWIRANPNFGNVGFTGGINSAILRYDGAAAVEPTTTQTTSTEPLNEVNLHPLVATAVPGSPVAGGVDLAINMAFNFNGTNFFINGASFTPPTVPVLLQIISGAQNAQDLLPSGSVYSLPSNADIEISFPATAAAPGAPHPFHLHGHAFAVVRSAGSTVYNYDNPIFRDVVSTGTPAAGDNVTIRFRTDNPGPWFLHCHIDFHLEAGFAVVFAEDIPDVASANPVPQAWSDLCPTYDALDPSDQ",
    )

    plate.assign_species(
        species=laccase, conc_unit="mmol / l", init_conc=0.000351, to="all"
    )

    # Define ABTS radical product
    radical = plate.add_reactant(**radical.to_dict())
    plate.assign_species(
        species=radical,
        conc_unit="mmol / l",
        init_conc=0,
        to="all",
        contributes_to_signal=False,
    )

    # Define ABTS substrate
    abts = plate.add_reactant(
        id="s0",
        name="ABTS",
        constant=False,
    )

plate_woTween.assign_species_from_spreadsheet(
    species=abts,
    conc_unit="mmol / l",
    path="laccase_kinetics/Laccase_kinetic_woTween_Rawdata.xlsx",
    sheet_name="Well assignment",
)

plate_Tween.assign_species_from_spreadsheet(
    species=abts,
    conc_unit="mmol / l",
    path="laccase_kinetics/Laccase_kinetic_wthTween_Rawdata.xlsx",
    sheet_name="Well assignment",
)

Assigned Trametes versicolor laccase to all wells.
Assigned ABTS radical to all wells.
Assigned Trametes versicolor laccase to all wells.
Assigned ABTS radical to all wells.


/Users/max/Documents/GitHub/software-driven-rdm/sdRDM/base/datamodel.py:324: UserWarning:

No 'URL' and 'Commit' specified. This model might not be re-usable.



In [12]:
for plate in plates:
    plate.blank_species(laccase, 420)

Mean absorption of Trametes versicolor laccase at 0.000351 mmol / l: -0.0405725
Blanked 64 wells containing Trametes versicolor laccase.
Mean absorption of Trametes versicolor laccase at 0.000351 mmol / l: -0.04699
Blanked 72 wells containing Trametes versicolor laccase.


In [13]:
enzml_woTween = plate_woTween.to_enzymeml(
    name="Laccase kinetics without Tween",
    detected_reactant=radical,
    wavelength=420,
    reactant_standard=standard,
    sort_measurements_by=abts,
    ignore_blank_status=True,
)

enzml_Tween = plate_Tween.to_enzymeml(
    name="Laccase kinetics with Tween",
    detected_reactant=radical,
    wavelength=420,
    sort_measurements_by=abts,
    reactant_standard=standard,
    ignore_blank_status=True,
)

Found 64 catalyzed wells
Found 72 catalyzed wells


## Parameter estimation

In [14]:
estimator_woTween, _ = Estimator.from_enzymeml(enzml_woTween, radical)
estimator_Tween, _ = Estimator.from_enzymeml(enzml_Tween, radical)

estimators = [estimator_woTween, estimator_Tween]

In [15]:
for estimator in estimators:
    estimator.add_reaction(
        id="r0",
        name="ABTS oxidation",
        educt=abts,
        product=radical,
        catalyst=laccase,
    )

    estimator.add_model(
        id="m0",
        name="Michaelis-Menten",
        equation="substrate = -(k_cat * catalyst * substrate) / (K_M + substrate)",
    )

### Visualize measurement data

In [16]:
estimator_woTween.visualize_data()

In [17]:
estimator_Tween.visualize_data()

Based on the visualization of the measurement data, measurements form wells E12, F12, G12, H12, E11, H11, G11, A11, B11, C11, D11, C4, E1, F1, G1, and H1 were excluded from kinetic analysis.

In [18]:
excluded_replicates = [
    "E12",
    "F12",
    "G12",
    "H12",
    "E11",
    "H11",
    "G11",
    "A11",
    "B11",
    "C11",
    "D11",
    "C4",
    "E1",
    "F1",
    "G1",
    "H1",
]

for replicate_id in excluded_replicates:
    estimator_Tween.remove_replicate(replicate_id)

In [19]:
estimator_Tween.visualize_data()

In [20]:
for estimator in estimators:
    min_time = 300
    estimator.fit_models(min_time=min_time)
    estimator.correlations()
    estimator.visualize(min_time=min_time)
    estimator.model_table()

Fitting Michaelis-Menten model: 100%|██████████| 1/1 [00:01<00:00,  1.18s/it]


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / s,mmol / l,1 / s / mmol / l,mmol / l,mmol / l,1 / s
Model,,,,,,,
Michaelis-Menten,-5602,2.437 ± 0.51 %,0.266 ± 3.33 %,,,,


[[-0.0056541  0.000351   0.0056541]]
[[0.05204917 0.000351   0.09795083]]
[[0.1810156 0.000351  0.1189844]]
[[0.3085291 0.000351  0.1414709]]
[[4.64127713e-01 3.51000000e-04 1.35872287e-01]]
[[6.00939786e-01 3.51000000e-04 1.49060214e-01]]
[[7.58119392e-01 3.51000000e-04 1.41880608e-01]]
[[8.4721104e-01 3.5100000e-04 1.5278896e-01]]
[[4.80823518e+00 3.51000000e-04 1.91764825e-01]]
[[9.74265986e+00 3.51000000e-04 2.57340136e-01]]
[[2.96689956e+01 3.51000000e-04 3.31004373e-01]]
[[3.96017427e+01 3.51000000e-04 3.98257303e-01]]
[[4.95868206e+01 3.51000000e-04 4.13179406e-01]]
[[5.95310746e+01 3.51000000e-04 4.68925368e-01]]
[[6.94957765e+01 3.51000000e-04 5.04223494e-01]]
[[7.94391575e+01 3.51000000e-04 5.60842478e-01]]


Fitting Michaelis-Menten model: 100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / s,mmol / l,1 / s / mmol / l,mmol / l,mmol / l,1 / s
Model,,,,,,,
Michaelis-Menten,-5498,1.986 ± 0.41 %,1.212 ± 2.59 %,,,,


[[-0.00991641  0.000351    0.00991641]]
[[0.27014246 0.000351   0.02985754]]
[[4.26057607e-01 3.51000000e-04 2.39423928e-02]]
[[6.75481842e-01 3.51000000e-04 7.45181581e-02]]
[[8.29705751e-01 3.51000000e-04 7.02942489e-02]]
[[9.20705945e-01 3.51000000e-04 7.92940546e-02]]
[[4.86809644e+00 3.51000000e-04 1.31903559e-01]]
[[9.80939349e+00 3.51000000e-04 1.90606510e-01]]
[[1.47833308e+01 3.51000000e-04 2.16669204e-01]]
[[1.97119396e+01 3.51000000e-04 2.88060430e-01]]
[[2.96566956e+01 3.51000000e-04 3.43304420e-01]]
[[3.96094918e+01 3.51000000e-04 3.90508205e-01]]
[[4.95567020e+01 3.51000000e-04 4.43298044e-01]]
[[5.94787611e+01 3.51000000e-04 5.21238936e-01]]
[[6.94334644e+01 3.51000000e-04 5.66535632e-01]]


In [21]:
reaction_system = estimator_woTween.get_reaction_system("Michaelis-Menten")
estimator_woTween.to_enzymeml(
    enzml_woTween,
    reaction_system=reaction_system,
    out_path=f"{enzml_woTween.name}.json",
)

EnzymeMLDocument(id='enzymemldocument0', name='Laccase kinetics without Tween', pubmedid=None, url=None, doi=None, created=datetime.datetime(2023, 12, 5, 14, 47, 57, 184290), modified=None, creators=[], vessels=[Vessel(id='plate0', name='MTP 96 well', volume=200.0, unit='ul', constant=True, uri=None, creator_id=None)], proteins=[Protein(id='p0', name='Trametes versicolor laccase', vessel_id='plate0', init_conc=None, constant=True, unit=None, uri=None, creator_id=None, sequence='MSRFHSLLAFVVASLTAVAHAGIGPVADLTITNAAVSPDGFSRQAVVVNGGTPGPLITGNMGDRFQLNVIDNLTNHTMLKSTSIHWHGFFQKGTNWADGPAFINQCPISSGHSFLYDFQVPDQAGTFWYHSHLSTQYCDGLRGPFVVYDPNDPAADLYDVDNDDTVITLVDWYHVAANVGPAFPLGADATLINGKGRSPSTTTADLSVISVTPGKRYRFRLVSLSCDPNYTFSIDGHNMTIIETDSINTAPLVVDSIQIFAAQRYSFVLEANQAVDNYWIRANPNFGNVGFTGGINSAILRYDGAAAVEPTTTQTTSTEPLNEVNLHPLVATAVPGSPVAGGVDLAINMAFNFNGTNFFINGASFTPPTVPVLLQIISGAQNAQDLLPSGSVYSLPSNADIEISFPATAAAPGAPHPFHLHGHAFAVVRSAGSTVYNYDNPIFRDVVSTGTPAAGDNVTIRFRTDNPGPWFLHCHIDFHLEAGFAVVFAEDIPDVASANPVPQAWSDLCPTYDALDP

In [22]:
reaction_system = estimator_Tween.get_reaction_system("Michaelis-Menten")
estimator_Tween.to_enzymeml(
    enzml_Tween,
    reaction_system=reaction_system,
    out_path=f"{enzml_Tween.name}.json",
)

EnzymeMLDocument(id='enzymemldocument1', name='Laccase kinetics with Tween', pubmedid=None, url=None, doi=None, created=datetime.datetime(2023, 12, 5, 14, 48, 0, 187910), modified=None, creators=[], vessels=[Vessel(id='plate0', name='MTP 96 well', volume=200.0, unit='ul', constant=True, uri=None, creator_id=None)], proteins=[Protein(id='p0', name='Trametes versicolor laccase', vessel_id='plate0', init_conc=None, constant=True, unit=None, uri=None, creator_id=None, sequence='MSRFHSLLAFVVASLTAVAHAGIGPVADLTITNAAVSPDGFSRQAVVVNGGTPGPLITGNMGDRFQLNVIDNLTNHTMLKSTSIHWHGFFQKGTNWADGPAFINQCPISSGHSFLYDFQVPDQAGTFWYHSHLSTQYCDGLRGPFVVYDPNDPAADLYDVDNDDTVITLVDWYHVAANVGPAFPLGADATLINGKGRSPSTTTADLSVISVTPGKRYRFRLVSLSCDPNYTFSIDGHNMTIIETDSINTAPLVVDSIQIFAAQRYSFVLEANQAVDNYWIRANPNFGNVGFTGGINSAILRYDGAAAVEPTTTQTTSTEPLNEVNLHPLVATAVPGSPVAGGVDLAINMAFNFNGTNFFINGASFTPPTVPVLLQIISGAQNAQDLLPSGSVYSLPSNADIEISFPATAAAPGAPHPFHLHGHAFAVVRSAGSTVYNYDNPIFRDVVSTGTPAAGDNVTIRFRTDNPGPWFLHCHIDFHLEAGFAVVFAEDIPDVASANPVPQAWSDLCPTYDALDPSDQ'